# Autologging with MLflow
MLflow can log metrics, parameters and artifacts (files) automatically. In this code, we will go over how this can be done.


In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.16.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


My credentials, subscription id, resource group and workspace info, are all stored in a file called config.py. You have to make sure that you have yours in that file.

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# My subscription id, resource group and workspace are all in the file below.
import config

In [3]:
ml_client = MLClient(
    DefaultAzureCredential(), config.subscription_id, config.resource_group, config.workspace
)

In [4]:
# We would like to create a folder for the script files

import os

script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)

In [5]:
%%writefile $script_folder/train-model-autolog.py
# import libraries
import mlflow
import argparse
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

def main(args):

    # enable autologging
    mlflow.autolog()

    # read data
    print(f'This is the path, haa {args.training_data}')
    df = get_data(args.training_data)

    # split data
    X_train, X_test, y_train, y_test = split_data(df)

    # train model
    model = train_model(args.reg_rate, X_train, X_test, y_train, y_test)

    # evaluate model
    eval_model(model, X_test, y_test)

# function that reads the data
def get_data(path):
    print("Reading data...")
    df = pd.read_csv(path)
    
    return df

# function that splits the data
def split_data(df):
    print("Splitting data...")
#     df = pd.get_dummies(df, columns = ['Sex'], dtype=int)
#     le = LabelEncoder()
#     df['Survived'] = le.fit_transform(df['Survived'])

    X = df.drop(['Survived'], axis=1)
    y = df['Survived']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

    return X_train, X_test, y_train, y_test

# function that trains the model
def train_model(reg_rate, X_train, X_test, y_train, y_test):
    mlflow.log_param("Regularization rate", reg_rate)
    print("Training model...")
    model = LogisticRegression(C=1/reg_rate, solver="liblinear").fit(X_train, y_train)

    return model

# function that evaluates the model
def eval_model(model, X_test, y_test):
    # calculate accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    print('Accuracy:', acc)

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)
    parser.add_argument("--reg_rate", dest='reg_rate',
                        type=float, default=0.01)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

Writing src/train-model-autolog.py


In [6]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

# configure job
data_asset = ml_client.data.get("titanic_ds4", version="1")

job = command(
    code="./src",
    command="python train-model-autolog.py --training_data ${{inputs.data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={"data": Input(path=data_asset.id,
                 type=AssetTypes.URI_FILE,
                 mode=InputOutputModes.RO_MOUNT
                 ),
            "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="sckaraman1",
    display_name="titanic-lives",
    experiment_name="titanic-lives-1",
    )

In [7]:
# Let's make sure that our code works first, before trying hyperparameters

returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.01 MBs): 100%|███████

Monitor your job at https://ml.azure.com/runs/bold_truck_05bsb4mt25?wsid=/subscriptions/a54b1e51-86a2-4073-b2a5-1a79c43cf955/resourcegroups/model_dep/workspaces/ml-workspace&tid=2e5df792-f2e7-43fc-954b-c3dd8d56a02d


In [10]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

job_name = returned_job.name
print(job_name)

bold_truck_05bsb4mt25


### Caution: You have to wait for the above job to finish before running the cell below

In [11]:
run_model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/model/",
    name="mlflow-titanic",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL,
)
# Uncomment after adding required details above
ml_client.models.create_or_update(run_model)

Model({'job_name': 'bold_truck_05bsb4mt25', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'mlflow-titanic', 'description': 'Model created from run.', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/a54b1e51-86a2-4073-b2a5-1a79c43cf955/resourceGroups/model_dep/providers/Microsoft.MachineLearningServices/workspaces/ml-workspace/models/mlflow-titanic/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/sckaraman1/code/Users/sckaraman', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f250bce9ae0>, 'serialize': <msrest.serialization.Serializer object at 0x7f250bce8580>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/a54b1e51-86a2-4073-b2a5-1a79c43cf955/resourceGroups/model_dep/workspaces/ml-workspace/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.bold_truck_05bsb4mt25/